<a href="https://colab.research.google.com/github/yankaSmetanka/bot_Hrytsyshyn/blob/main/laba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Підвантаження потрібних бібліотек
!pip install pymorphy3
!pip install pymorphy3_dicts_uk
!pip install tokenize_uk

import re
import math
import random
import pymorphy3
import pandas as pd
from google.colab import files

import pymorphy3_dicts_uk
morph = pymorphy3.MorphAnalyzer(lang='uk')

import tokenize_uk as tu
import sqlite3

import collections
from collections import defaultdict

import numpy as np

In [ ]:
# Конвертація тексту в txt

def textDetails(text):
  file_name = re.search(r'.+\.(.+)\.txt', text).group(1)
  file_type = re.search(r'(.+?)\..+', text).group(1)
  return file_name, file_type


def fileToText(file):
  with open(file, 'r', encoding='utf-8') as f:
      return f.read()

# Розбиття на речення

def textToSent(text):
  text_by_parag = re.sub(r"\d+\.\d*", '', text).split('\n')
  text_by_sent = {}
  i=1

  for parag in text_by_parag:
    for sent in tu.tokenize_sents(parag):
      text_by_sent[i] = sent
      i+=1

  return text_by_sent

# Розбиття тексту на слова, визначення для кожного слова його нормалізованої форми та частини мови
def textToWords(text):
  text_by_words = {}

  text = re.sub(r"[^а-яА-Яєґії’]", ' ', text).lower().split()

  for w in text:
      text_by_words[w] = (morph.parse(w)[0].normal_form, morph.parse(w)[0].tag.POS)

  return text_by_words

In [19]:
# Створення табличок
conn = sqlite3.connect('corpus_db.db')
c = conn.cursor()

c.execute('''DROP TABLE IF EXISTS texts''')
c.execute('''DROP TABLE IF EXISTS sentences''')
c.execute('''DROP TABLE IF EXISTS lemmas''')
c.execute('''DROP TABLE IF EXISTS words''')
c.execute('''DROP TABLE IF EXISTS poss''')

c.execute('''CREATE TABLE IF NOT EXISTS texts (
    text_id INTEGER PRIMARY KEY,
    text_name TEXT,
    text_type VARCHAR(20))
    ''')

c.execute('''CREATE TABLE IF NOT EXISTS sentences (
    sent_id INTEGER PRIMARY KEY,
    sent TEXT,
    sent_num INTEGER,
    text_id INTEGER,
    FOREIGN KEY (text_id) REFERENCES texts(text_id))
    ''')

c.execute('''CREATE TABLE IF NOT EXISTS lemmas (
    lemma_id INTEGER PRIMARY KEY,
    lemma VARCHAR(50) UNIQUE,
    pos VARCHAR(10))
    ''')

c.execute('''CREATE TABLE IF NOT EXISTS words (
    word_id INTEGER PRIMARY KEY,
    word VARCHAR(50),
    lemma VARCHAR(50),
    pos VARCHAR(10),
    sent_id INTEGER,
    text_id INTEGER,
    FOREIGN KEY (sent_id) REFERENCES sentences(sent_id),
    FOREIGN KEY (text_id) REFERENCES texts(text_id))
    ''')

c.execute('''CREATE TABLE IF NOT EXISTS poss (
    pos_id INTEGER PRIMARY KEY,
    pos VARCHAR(10),
    abs_freq_fict INTEGER,
    abs_freq_non_fict INTEGER,
    abs_freq_all INTEGER)
    ''')

conn.commit()

In [20]:
# Відкриваємо обрані тексти і заповнюємо таблички

uploaded_files = files.upload()
conn = sqlite3.connect('corpus_db.db')
c = conn.cursor()

text_lib = ['fiction.letmypeoplego.txt', 'nonfiction.praviladorochnogoruhu2013.txt']


text_id = 1
sent_id = 1
word_id = 1
lemma_id = 1

for text in text_lib:
  text_name = textDetails(text)[0]
  text_type = textDetails(text)[1]
  c.execute('''INSERT INTO texts VALUES (?, ?, ?)''',
              (text_id, text_name, text_type))

  for sent_num, sent in textToSent(fileToText(text)).items():
    c.execute('''INSERT INTO sentences VALUES (?, ?, ?, ?)''',
              (sent_id, sent, sent_num, text_id))

    for word, pos_lemma in textToWords(sent).items():
      #print(word)
      #print(pos_lemma[0])
      pos_lemma_clean = re.sub(r"'", '’', pos_lemma[0])
      c.execute(f"SELECT lemma_id FROM lemmas WHERE lemma = '{pos_lemma_clean}'")
      if c.fetchone() == None:
          c.execute('''INSERT INTO lemmas VALUES (?, ?, ?)''', (lemma_id, pos_lemma_clean, pos_lemma[1]))
          lemma_id += 1

      c.execute('''INSERT INTO words VALUES (?, ?, ?, ?, ?, ?)''',
                  (word_id, word, pos_lemma_clean, pos_lemma[1], sent_id, text_id))

      word_id += 1

    sent_id += 1

  text_id += 1
conn.commit()

Saving fiction.letmypeoplego.txt to fiction.letmypeoplego.txt
Saving nonfiction.praviladorochnogoruhu2013.txt to nonfiction.praviladorochnogoruhu2013.txt


In [24]:
# Підвантажуємо файли та формуємо ЧС частин мови

conn = sqlite3.connect('corpus_db.db')
c = conn.cursor()

pos_id = 1

c.execute('''SELECT text_id FROM texts WHERE text_type = "fiction"''')
text_id_fict = c.fetchall()

c.execute(f"SELECT pos FROM words WHERE text_id = ?", (text_id_fict[0][0],))
poss_fict_list = list(c.fetchall())
for i in poss_fict_list:
  if i[0] == None:
    poss_fict_list.pop(poss_fict_list.index(i))

for pos in poss_fict_list:
  c.execute(f'''SELECT pos_id FROM poss WHERE pos = ?''', (pos[0],))
  if c.fetchone() == None:
    c.execute('''INSERT INTO poss VALUES (?, ?, ?, ?, ?)''',
                (pos_id, pos[0], 1, 0, 1))
    pos_id += 1
  else:
    c.execute('''UPDATE poss SET abs_freq_fict = abs_freq_fict + 1 WHERE pos = ?''', (pos[0],))
    c.execute('''UPDATE poss SET abs_freq_all = abs_freq_all + 1 WHERE pos = ?''', (pos[0],))



c.execute('''SELECT text_id FROM texts WHERE text_type = "nonfiction"''')
text_id_non_fict = c.fetchall()

c.execute(f"SELECT pos FROM words WHERE text_id = ?", (text_id_non_fict[0][0],))
poss_non_fict_list = list(c.fetchall())
for i in poss_non_fict_list:
  if i[0] == None:
    poss_non_fict_list.pop(poss_non_fict_list.index(i))

for pos in poss_non_fict_list:
  c.execute(f'''SELECT pos_id FROM poss WHERE pos = ?''', (pos[0],))
  if c.fetchone() == None:
    c.execute('''INSERT INTO poss VALUES (?, ?, ?, ?, ?)''',
                (pos_id, pos[0], 1, 0, 1))
    pos_id += 1
  else:
    c.execute('''UPDATE poss SET abs_freq_non_fict = abs_freq_non_fict + 1 WHERE pos = ?''', (pos[0],))
    c.execute('''UPDATE poss SET abs_freq_all = abs_freq_all + 1 WHERE pos = ?''', (pos[0],))


c.execute('''SELECT * FROM poss''')
result = c.fetchall()

for row in result:
    print(row)


(1, 'NOUN', 10559, 10769, 21328)
(2, 'ADJF', 3599, 4027, 7626)
(3, 'PREP', 1501, 1649, 3150)
(4, 'NPRO', 2938, 1031, 3969)
(5, 'ADVB', 1852, 948, 2800)
(6, 'VERB', 3904, 2040, 5944)
(7, 'PRCL', 1816, 974, 2790)
(8, 'CONJ', 1823, 1764, 3587)
(9, 'GRND', 197, 50, 247)
(10, 'NUMR', 165, 134, 299)
(11, None, 1, 0, 1)
(12, None, 1, 0, 1)
(13, None, 1, 0, 1)
(14, None, 1, 0, 1)
(15, None, 1, 0, 1)
(16, None, 1, 0, 1)
(17, None, 1, 0, 1)
(18, None, 1, 0, 1)
(19, None, 1, 0, 1)
(20, None, 1, 0, 1)
(21, None, 1, 0, 1)
(22, None, 1, 0, 1)
(23, None, 1, 0, 1)
(24, None, 1, 0, 1)
(25, None, 1, 0, 1)
(26, None, 1, 0, 1)
(27, None, 1, 0, 1)
(28, None, 1, 0, 1)
(29, None, 1, 0, 1)
(30, None, 1, 0, 1)
(31, None, 1, 0, 1)
(32, None, 1, 0, 1)
(33, None, 1, 0, 1)
(34, None, 1, 0, 1)
(35, None, 1, 0, 1)
(36, None, 1, 0, 1)
(37, None, 1, 0, 1)
(38, None, 1, 0, 1)
(39, None, 1, 0, 1)
(40, None, 1, 0, 1)
(41, None, 1, 0, 1)
(42, None, 1, 0, 1)


In [27]:
# Дозаповнення інших табличок

conn = sqlite3.connect('corpus_db.db')
c = conn.cursor()

print('\nTEXTS:')
c.execute("SELECT * FROM texts")
result = c.fetchall()

for row in result:
    print(row)

print('\nSENTENCES:')
c.execute("SELECT * FROM sentences")
result = c.fetchall()

for row in result:
    print(row)

print('\nWORDS:')
c.execute("SELECT * FROM words")
result = c.fetchall()

for row in result:
  print(row)

print('\nLEMMAS:')
c.execute("SELECT * FROM lemmas")
result = c.fetchall()

for row in result:
    print(row)

print('\nPOS:')
c.execute("SELECT * FROM poss")
result = c.fetchall()

for row in result:
    print(row)

conn.close()

Показано результат, скорочений до останніх рядків (5000).
(33612, 'засобів', 'засіб', 'NOUN', 1390, 2)
(33613, 'не', 'не', 'PRCL', 1390, 2)
(33614, 'повинні', 'повинний', 'ADJF', 1390, 2)
(33615, 'перешкоджати', 'перешкоджати', 'VERB', 1390, 2)
(33616, 'їхній', 'їхній', 'NPRO', 1390, 2)
(33617, 'роботі', 'робот', 'NOUN', 1390, 2)
(33618, 'обов', 'обов', 'NOUN', 1391, 2)
(33619, 'язки', 'язка', 'NOUN', 1391, 2)
(33620, 'і', 'і', 'CONJ', 1391, 2)
(33621, 'права', 'право', 'NOUN', 1391, 2)
(33622, 'пішоходів', 'пішохід', 'NOUN', 1391, 2)
(33623, 'пішоходи', 'пішохід', 'NOUN', 1393, 2)
(33624, 'повинні', 'повинний', 'ADJF', 1393, 2)
(33625, 'рухатися', 'рухатися', 'VERB', 1393, 2)
(33626, 'по', 'по', 'PREP', 1393, 2)
(33627, 'тротуарах', 'тротуар', 'NOUN', 1393, 2)
(33628, 'і', 'і', 'CONJ', 1393, 2)
(33629, 'пішохідних', 'пішохідний', 'ADJF', 1393, 2)
(33630, 'доріжках', 'доріжка', 'NOUN', 1393, 2)
(33631, 'тримаючись', 'тримаючись', 'GRND', 1393, 2)
(33632, 'правого', 'правий', 'ADJF', 13

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Показано результат, скорочений до останніх рядків (5000).
(4016, 'згодом', 'ADVB')
(4017, 'пересістися', 'VERB')
(4018, 'золотий', 'NOUN')
(4019, 'гомін', 'NOUN')
(4020, 'підбадьоритися', 'VERB')
(4021, 'вгледівши', 'GRND')
(4022, 'сергій', 'NOUN')
(4023, 'жадан', 'NOUN')
(4024, 'комендант', 'NOUN')
(4025, 'ага', 'PRCL')
(4026, 'гром', 'NOUN')
(4027, 'блискавка', 'NOUN')
(4028, 'анатолій', 'NOUN')
(4029, 'ульянов', 'NOUN')
(4030, 'публ', 'NOUN')
(4031, 'чнити', 'VERB')
(4032, 'нтелектуал', 'NOUN')
(4033, 'н', None)
(4034, 'семінар', 'NOUN')
(4035, 'фулбрайт', 'NOUN')
(4036, 'круглий', 'ADJF')
(4037, 'різножанровий', 'ADJF')
(4038, 'інтелігентський', 'ADJF')
(4039, 'зібрання', 'NOUN')
(4040, 'пореволюційний', 'ADJF')
(4041, 'фулбрайтівський', 'ADJF')
(4042, 'дяка', 'NOUN')
(4043, 'шана', 'NOUN')
(4044, 'університет', 'NOUN')
(4045, 'вперто', 'ADVB')
(4046, 'зрікатися', 'VERB')
(4047, 'сан', 'NOUN')
(4048, 'обійтися', 'VERB')
(4049, 'славнозвісний', 'ADJF')
(4050, 'валянок', 'NOUN')
(405